In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
combined_df = pd.read_csv('../cleandata/combined_spotify_data.csv')

In [4]:
import altair as alt
top10_genres = (
    combined_df.groupby("playlist_genre")["track_popularity"]
        .mean()
        .sort_values(ascending=False)
        .head(10)
)
genres_df = combined_df[combined_df["playlist_genre"].isin(top10_genres.index)]
genres_df = genres_df[["playlist_genre", "track_popularity"]]

legend_color = [
    "#8dd3c7", "#ffffb3", "#bebada", "#fb8072", "#80b1d3",
    "#fdb462", "#b3de69", "#fccde5", "#664bdd", "black"
]

selection = alt.selection_point(fields=["playlist_genre"])
slider = alt.binding_range(min=0, max=100, step=5, name="Max Popularity: ")
var = alt.param(value=100, bind=slider)

chart = (
    alt.Chart(genres_df)
    .transform_filter(
        alt.datum.track_popularity <= var
    )
    .transform_density(
        "track_popularity",
        as_=["track_popularity", "density"],
        groupby=["playlist_genre"]
    )
    .mark_line()
    .encode(
        x="track_popularity:Q",
        y="density:Q",
        color=alt.condition(
            selection,
            alt.Color("playlist_genre:N", scale=alt.Scale(range=legend_color)),
            alt.value("lightgray")
        )
    )
    .add_params(selection, var)
    .properties(
        width=800,
        height=400,
        title="Distribution Of Track Popularity Across The Top 10 Spotify Genres"
    )
)

chart


alt.Chart(...)

In [6]:
df =pd.read_csv('../cleandata/combined_spotify_data.csv')
top10 = (
    df.groupby("playlist_genre")["track_popularity"]
        .mean()
        .sort_values(ascending=False)
        .head(10)
        .index
)

df10 = df[df["playlist_genre"].isin(top10)]


step = 50
overlap = 1

alt.Chart(df10, height=step, width=700).transform_joinaggregate(
    mean_value='mean(danceability)', groupby=['playlist_genre']
).transform_bin(
    ['bin_max', 'bin_min'], 'danceability'
).transform_aggregate(
    value='count()',
    groupby=['playlist_genre', 'mean_value', 'bin_min', 'bin_max']
).transform_impute(
    impute='value',
    groupby=['playlist_genre', 'mean_value'],
    key='bin_min',
    value=0
).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='lightgray',
    strokeWidth=0.5
).encode(
    alt.X('bin_min:Q').bin('binned').title('danceability (0–1)'),
    alt.Y('value:Q').axis(None).scale(range=[step, -step * overlap]),
    alt.Fill('playlist_genre:N').legend(None).scale(
        range=[
            "#0588df", "#ffffb3", "#bebada", "#fb8072", "#fd2856",
            "#47caa3", "#b3de69", "#fccde5", "#664bdd", "grey"
        ]
    )
).facet(
    row=alt.Row('playlist_genre:N').title(None).header(labelAngle=0, labelAlign='left')
).properties(
    title='Distribution of danceability by Top 10 Playlist Genres',
    bounds='flush'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)

alt.FacetChart(...)